In [25]:
import torch

torch.set_default_dtype(torch.float64)

class NormalDistribution:
    def __init__(self, mean, logvar, v=None, r=None, A=None):
        """
        :param mean: mu in the paper
        :param logvar: \Sigma in the paper
        :param v:
        :param r:
        if A is not None then covariance matrix = A \Sigma A^T, where A = I + v^T r
        else the covariance matrix is simply diag(logvar.exp())
        """
        self.mean = mean
        self.logvar = logvar
        self.v = v
        self.r = r

        sigma = torch.diag_embed(torch.exp(logvar))
        if A is None:
            self.cov = sigma
        else:
            self.cov = A.bmm(sigma.bmm(A.transpose(1, 2)))


    @staticmethod
    def KL_divergence(q_z_next_pred, q_z_next):
        """
        :param q_z_next_pred: q(z_{t+1} | z_bar_t, q_z_t, u_t) using the transition
        :param q_z_next: q(z_t+1 | x_t+1) using the encoder
        :return: KL divergence between two distributions
        """
        mu_0 = q_z_next_pred.mean
        mu_1 = q_z_next.mean
        sigma_0 = torch.exp(q_z_next_pred.logvar)
        sigma_1 = torch.exp(q_z_next.logvar)
        v = q_z_next_pred.v
        r = q_z_next_pred.r
        k = float(q_z_next_pred.mean.size(1))

        sum = lambda x: torch.sum(x, dim=1)

        KL = 0.5 * torch.mean(sum((sigma_0 + 2*sigma_0*v*r) / sigma_1)
                              + sum(r.pow(2) * sigma_0) * sum(v.pow(2) / sigma_1)
                              + sum(torch.pow(mu_1-mu_0, 2) / sigma_1) - k
                              + 2 * (sum(q_z_next.logvar - q_z_next_pred.logvar) - torch.log(1 + sum(v*r)))
                              )
        return KL

In [26]:
# add input shape and output shape to the network at each layer if input =(batch_size,4)
from torch import nn

import pdb

from torch.autograd import Variable
import torch
from torch import nn


torch.set_default_dtype(torch.float64)

def weights_init(m):
    if type(m) in [nn.Conv2d, nn.Linear, nn.ConvTranspose2d]:
        torch.nn.init.orthogonal_(m.weight)

class Encoder(nn.Module):
    def __init__(self, net, obs_dim, z_dim):
        super(Encoder, self).__init__()
        self.net = net
        self.net.apply(weights_init)
        self.img_dim = obs_dim
        self.z_dim = z_dim

    def forward(self, x):
        """
        :param x: observation
        :return: the parameters of distribution q(z|x)
        """
        return self.net(x).chunk(2, dim = 1) # first half is mean, second half is logvar

class Decoder(nn.Module):
    def __init__(self, net, z_dim, obs_dim):
        super(Decoder, self).__init__()
        self.net = net
        self.net.apply(weights_init)
        self.z_dim = z_dim
        self.obs_dim = obs_dim

    def forward(self, z):
        """
        :param z: sample from q(z|x)
        :return: reconstructed x
        """
        return self.net(z)

class Transition(nn.Module):
    def __init__(self, net, z_dim, u_dim):
        super(Transition, self).__init__()
        self.net = net  # network to output the last layer before predicting A_t, B_t and o_t
        self.net.apply(weights_init)
        #pdb.set_trace()
        self.h_dim = self.net[-3].out_features
        self.z_dim = z_dim
        self.u_dim = u_dim

        self.fc_A = nn.Sequential(
            nn.Linear(self.h_dim, self.z_dim * 2), # v_t and r_t
            nn.Sigmoid()
        )
        self.fc_A.apply(weights_init)

        self.fc_B = nn.Linear(self.h_dim, self.z_dim * self.u_dim)
        torch.nn.init.orthogonal_(self.fc_B.weight)
        self.fc_o = nn.Linear(self.h_dim, self.z_dim)
        torch.nn.init.orthogonal_(self.fc_o.weight)
    def forward(self, z_bar_t, q_z_t, u_t):
        """
        :param z_bar_t: the reference point
        :param Q_z_t: the distribution q(z|x)
        :param u_t: the action taken
        :return: the predicted q(z^_t+1 | z_t, z_bar_t, u_t)
        """
        h_t = self.net(z_bar_t)
        B_t = self.fc_B(h_t)
        o_t = self.fc_o(h_t)
 


        v_t, r_t = self.fc_A(h_t).chunk(2, dim=1)

        v_t = torch.unsqueeze(v_t, dim=-1)
        r_t = torch.unsqueeze(r_t, dim=-2)

        A_t = torch.eye(self.z_dim).repeat(z_bar_t.size(0), 1, 1).to('cpu') + torch.bmm(v_t, r_t)

        B_t = B_t.view(-1, self.z_dim, self.u_dim)

        mu_t = q_z_t.mean

        mean1 = A_t.bmm(mu_t.unsqueeze(-1)).squeeze(-1) 
        mean2= B_t.bmm(u_t.unsqueeze(-1).unsqueeze(-1)).squeeze(-1) + o_t
        mean=mean1+mean2


        return mean, NormalDistribution(mean, logvar=q_z_t.logvar, v=v_t.squeeze(), r=r_t.squeeze(), A=A_t)

class springEncoder(Encoder):
    def __init__(self, obs_dim = 2, z_dim = 2):
        net = nn.Sequential(
            nn.Linear(obs_dim, 150),
            nn.BatchNorm1d(150),
            nn.ReLU(),

            nn.Linear(150, 150),
            nn.BatchNorm1d(150),
            nn.ReLU(),

            nn.Linear(150, 150),
            nn.BatchNorm1d(150),
            nn.ReLU(),

            nn.Linear(150, 150),
            nn.BatchNorm1d(150),
            nn.ReLU(),

            nn.Linear(150, 150),
            nn.BatchNorm1d(150),
            nn.ReLU(),

            nn.Linear(150, z_dim * 2)
        )
        super(springEncoder, self).__init__(net, obs_dim, z_dim)

class springDecoder(Decoder):
    def __init__(self, z_dim = 2, obs_dim = 2):
        net = nn.Sequential(
            nn.Linear(z_dim, 150),
            nn.BatchNorm1d(150),
            nn.ReLU(),

            nn.Linear(150, 150),
            nn.BatchNorm1d(150),
            nn.ReLU(),

            nn.Linear(150, 150),
            nn.BatchNorm1d(150),
            nn.ReLU(),

            nn.Linear(150, 150),
            nn.BatchNorm1d(150),
            nn.ReLU(),

            nn.Linear(150, 150),
            nn.BatchNorm1d(150),
            nn.ReLU(),
            nn.Linear(150, obs_dim),
            nn.Sigmoid()
        )
        super(springDecoder, self).__init__(net, z_dim, obs_dim)

class springTransition(Transition):
    def __init__(self, z_dim = 2, u_dim = 1):
        net = nn.Sequential(
            nn.Linear(z_dim, 100),
            nn.BatchNorm1d(100),
            nn.ReLU(),

            nn.Linear(100, 100),
            nn.BatchNorm1d(100),
            nn.ReLU(),

            nn.Linear(100, 100),
            nn.BatchNorm1d(100),
            nn.ReLU(),

            nn.Linear(100, 100),
            nn.BatchNorm1d(100),
            nn.ReLU(),

            nn.Linear(100, 100),
            nn.BatchNorm1d(100),

            nn.Sigmoid()
        )
        super(springTransition, self).__init__(net, z_dim, u_dim)



class E2C(nn.Module):
    def __init__(self, obs_dim, z_dim, u_dim, env = 'experimental'):
        super(E2C, self).__init__()

        self.obs_dim = obs_dim
        self.z_dim = z_dim
        self.u_dim = u_dim

        self.encoder = springEncoder(obs_dim=obs_dim, z_dim=z_dim)
        # self.encoder.apply(init_weights)
        self.decoder = springDecoder(z_dim=z_dim, obs_dim=obs_dim)
        # self.decoder.apply(init_weights)
        self.trans = springTransition(z_dim=z_dim, u_dim=u_dim)
        # self.trans.apply(init_weights)

    def encode(self, x):
        """
        :param x:
        :return: mean and log variance of q(z | x)
        """
        #print(x.shape)
        return self.encoder(x)

    def decode(self, z):
        """
        :param z:
        :return: bernoulli distribution p(x | z)
        """
        return self.decoder(z)

    def transition(self, z_bar, q_z, u):
        """

        :return: samples z_hat_next and Q(z_hat_next)
        """
        return self.trans(z_bar, q_z, u)

    def reparam(self, mean, logvar,is_train=True):
        sigma = (logvar / 2).exp()
        epsilon = torch.randn_like(sigma)
        if(is_train):
          #print('yes')
          return mean + torch.mul(epsilon, sigma)
          
        else:
          print('yes') 
          return mean

    def forward(self, x, u, x_next,is_train=True):
        #pdb.set_trace()
        mu, logvar = self.encode(x)
        z = self.reparam(mu, logvar,is_train)
        q_z = NormalDistribution(mu, logvar)

        x_recon = self.decode(z)

        z_next, q_z_next_pred = self.transition(z, q_z, u)
        x_next_pred = self.decode(z_next)
        mu_next, logvar_next = self.encode(x_next)
        q_z_next = NormalDistribution(mean=mu_next, logvar=logvar_next)

        return x_recon, x_next_pred, q_z, q_z_next_pred, q_z_next,z

    def predict(self, x, u):
        mu, logvar = self.encoder(x)
        z = self.reparam(mu, logvar)
        q_z = NormalDistribution(mu, logvar)

        z_next, q_z_next_pred = self.transition(z, q_z, u)

        x_next_pred = self.decode(z_next)
        return x_next_pred,z_next

In [27]:
import os
from os import path
import numpy as np
import json
from torchvision.transforms import ToTensor
from torch.utils.data import Dataset
from tqdm import tqdm
import pickle
import torch
class experimental(Dataset):
    def __init__(self, dir):
        self.dir = dir
        with open(path.join(dir, 'data.json')) as f:
            self._data = json.load(f)
        self._process()
    def __len__(self):
        return len(self._data['samples'])
    def __getitem__(self, index):
        return self._processed[index]

    def _process(self):
        preprocessed_file = os.path.join(self.dir, 'processed.pkl')
        if not os.path.exists(preprocessed_file):
            processed = []
            for sample in tqdm(self._data['samples'], desc='processing data'):
                before = sample['before']
                after = sample['after']
                control = sample['control']

                processed.append((np.array(before), np.array(control), np.array(after)))

            with open(preprocessed_file, 'wb') as f:
                pickle.dump(processed, f)
            self._processed = processed
        else:
            with open(preprocessed_file, 'rb') as f:
                self._processed = pickle.load(f)

In [28]:
#from tensorboardX import SummaryWriter
import torch.optim as optim
from torch.utils.data import DataLoader
import matplotlib.pyplot as plt
import argparse
import sys
import torch
import numpy as np
import os
import pdb
from torchsummary import summary

torch.set_default_dtype(torch.float64)

device = torch.device("cuda")


datasets = {'experimental':experimental}
settings = {'experimental':(8,8,1)}


num_eval=10
def compute_loss(x, x_next, q_z_next, x_recon, x_next_pred, q_z, q_z_next_pred, lamda):
    # lower-bound loss
    recon_term = -torch.mean(torch.sum(x * torch.log(1e-5 + x_recon)
                                       + (1 - x) * torch.log(1e-5 + 1 - x_recon), dim=1))
    pred_loss = -torch.mean(torch.sum(x_next * torch.log(1e-5 + x_next_pred)
                                      + (1 - x_next) * torch.log(1e-5 + 1 - x_next_pred), dim=1))

    kl_term = - 0.5 * torch.mean(torch.sum(1 + q_z.logvar - q_z.mean.pow(2) - q_z.logvar.exp(), dim = 1))

    #lower_bound = recon_term + pred_loss + kl_term
    lower_bound=recon_term + pred_loss + kl_term
    

    # consistency loss
    consis_term = NormalDistribution.KL_divergence(q_z_next_pred, q_z_next)
    return lower_bound + lamda * consis_term,recon_term,pred_loss,kl_term

def train(model, train_loader, lam, optimizer,scheduler):
    model.train()
    avg_loss = 0.0
    rec_loss = 0.0
    pred_loss = 0.0
    kl_loss = 0.0

    num_batches = len(train_loader)
    for i, (x, u, x_next) in enumerate(train_loader, 0):
        x = x.view(-1, model.obs_dim).double().to(device)
        u = u.double().to(device)
        x_next = x_next.view(-1, model.obs_dim).double().to(device)
        optimizer.zero_grad()
        x_recon, x_next_pred, q_z, q_z_next_pred, q_z_next ,_= model(x, u, x_next)

        loss,rec,pred,kl = compute_loss(x, x_next, q_z_next, x_recon, x_next_pred, q_z, q_z_next_pred, lam)
        rec_loss += rec.item()
        pred_loss += pred.item()
        #kl_loss += kl.item()
        avg_loss += loss.item()
        loss.backward()
        optimizer.step()
        scheduler.step()

    return avg_loss / num_batches, pred_loss / num_batches, kl_loss / num_batches, rec_loss / num_batches

def compute_log_likelihood(x, x_recon, x_next, x_next_pred):
    loss_1 = -torch.mean(torch.sum(x * torch.log(1e-5 + x_recon)
                                   + (1 - x) * torch.log(1e-5 + 1 - x_recon), dim=1))
    loss_2 = -torch.mean(torch.sum(x_next * torch.log(1e-5 + x_next_pred)
                                   + (1 - x_next) * torch.log(1e-5 + 1 - x_next_pred), dim=1))
    return loss_1, loss_2

def evaluate(model, test_loader):
    model.eval()
    num_batches = len(test_loader)
    state_loss, next_state_loss = 0., 0.
    with torch.no_grad():
        for x, u, x_next in test_loader:
            x = x.view(-1, model.obs_dim).double().to(device)
            u = u.double().to(device)
            x_next = x_next.view(-1, model.obs_dim).double().to(device)

            x_recon, x_next_pred, q_z, q_z_next_pred, q_z_next ,_= model(x, u, x_next)
            loss_1, loss_2 = compute_log_likelihood(x, x_recon, x_next, x_next_pred)
            state_loss += loss_1
            next_state_loss += loss_2

    return state_loss.item() / num_batches, next_state_loss.item() / num_batches

# code for visualizing the training process
def predict_x_next(model, env, num_eval):
    # frist sample a true trajectory from the environment
    sampler = samplers[env]
    state_samples, sampled_data = sampler.sample(num_eval)

    # use the trained model to predict the next observation
    predicted = []
    for x, u, x_next in sampled_data:
        x_reshaped = x.reshape(-1)
        x_reshaped = torch.from_numpy(x_reshaped).double().unsqueeze(dim=0).to(device)
        u = torch.from_numpy(u).double().unsqueeze(dim=0).to(device)
        with torch.no_grad():
            x_next_pred = model.predict(x_reshaped, u)
        predicted.append(x_next_pred.squeeze().cpu().numpy().reshape(sampler.width, sampler.height))
    true_x_next = [data[-1] for data in sampled_data]
    return true_x_next, predicted





In [11]:

# write a training loop which takes in the new dataset encodes it using old weights then learns a new tranisition matrix the transition and decodes it using the new weights 

env_name ='experimental'
propor = 0.9
batch_size = 1028
lr = 0.0001
weight_decay = 0.00001
lam = 0.25
epoches = 3000
iter_save = 100
seed =0
device='cpu'
np.random.seed(seed)
torch.manual_seed(seed)

dataset = datasets[env_name]('/Users/avi/Desktop/Food_GVAE-master/E2C_spring_mass/Sim_data/data_2' )
train_set, test_set = dataset[:int(len(dataset) * propor)], dataset[int(len(dataset) * propor):]
train_loader = DataLoader(train_set, batch_size=batch_size, shuffle=True, drop_last=False)
test_loader = DataLoader(test_set, batch_size=batch_size, shuffle=False, drop_last=False)
obs_dim, z_dim, u_dim = settings[env_name]
model1 = E2C(obs_dim=obs_dim, z_dim=z_dim, u_dim=u_dim, env=env_name).to(device)
#load weights to model1
model1.load_state_dict(torch.load('/Users/avi/Desktop/Food_GVAE-master/E2C_spring_mass/Sim_data/model_weights.pt'))
#freeze weights for model1 decoder and encoder
for param in model1.encoder.parameters():
    param.requires_grad = False
for param in model1.decoder.parameters():
    param.requires_grad = False


optimizer = optim.Adam(model1.parameters(), betas=(0.9, 0.999), eps=1e-8, lr=lr, weight_decay=weight_decay)
scheduler = torch.optim.lr_scheduler.CosineAnnealingWarmRestarts(optimizer, 20)

beta_start = 0.1
beta_end = 1.0
beta_steps = 10000
beta_step = (beta_end - beta_start) / beta_steps
beta = beta_start
for i in range(epoches):
    avg_loss,pred_loss,kl_loss,rec_loss = train(model1, train_loader, beta, optimizer,scheduler)
    print('Epoch %d' % i)
    print("Training loss: %f" % (avg_loss))
    beta = min(beta + beta_step, beta_end)
    #print("rec loss: %f" % (rec_loss))
    #print("pred loss: %f" % (pred_loss))
    #print("KL loss: %f" % (kl_loss))

    # evaluate on test set
    #state_loss, next_state_loss = evaluate(model1, test_loader)
    #print('State loss: ' + str(state_loss))
    #print('Next state loss: ' + str(next_state_loss))



Epoch 0
Training loss: 10.413281
Epoch 1
Training loss: 9.823708
Epoch 2
Training loss: 9.276559
Epoch 3
Training loss: 8.869634
Epoch 4
Training loss: 8.513844
Epoch 5
Training loss: 8.134258
Epoch 6
Training loss: 7.882627
Epoch 7
Training loss: 7.801052
Epoch 8
Training loss: 7.495729
Epoch 9
Training loss: 7.434354
Epoch 10
Training loss: 7.186022
Epoch 11
Training loss: 6.958961
Epoch 12
Training loss: 6.875355
Epoch 13
Training loss: 6.797355
Epoch 14
Training loss: 6.669731
Epoch 15
Training loss: 6.528584
Epoch 16
Training loss: 6.430170
Epoch 17
Training loss: 6.319764
Epoch 18
Training loss: 6.209154
Epoch 19
Training loss: 6.168400
Epoch 20
Training loss: 6.091222
Epoch 21
Training loss: 5.968618
Epoch 22
Training loss: 5.958766
Epoch 23
Training loss: 5.826901
Epoch 24
Training loss: 5.789562
Epoch 25
Training loss: 5.730983
Epoch 26
Training loss: 5.650877
Epoch 27
Training loss: 5.608411
Epoch 28
Training loss: 5.536871
Epoch 29
Training loss: 5.469975
Epoch 30
Training l

KeyboardInterrupt: 

In [29]:

dataset = datasets[env_name]('/Users/avi/Desktop/Food_GVAE-master/E2C_spring_mass/Sim_data/data_2' )
train_set, test_set = dataset[:int(len(dataset) * propor)], dataset[int(len(dataset) * propor):]
train_loader = DataLoader(train_set, batch_size=batch_size, shuffle=True, drop_last=False)
model1 = E2C(obs_dim=obs_dim, z_dim=z_dim, u_dim=u_dim, env=env_name)
#load weights to model1
model1.load_state_dict(torch.load('/Users/avi/Desktop/Food_GVAE-master/E2C_spring_mass/Sim_data/model_weights.pt'))
first_batch = next(iter(train_loader))
x=first_batch[0]
u=first_batch[1]
x_next=first_batch[2]
with torch.no_grad():
  device='cpu'
  x = x.view(-1, model1.obs_dim).double().to(device)
  x_next = x_next.view(-1, model1.obs_dim).double().to(device)
  mu,var= model1.encode(x)
  mu2,var2= model1.enocde(x_next)
  #x_next_pred=x_next_pred.cpu()
  #x=x.cpu()
  #x=x.detach().numpy()

  #x_next_pred=x_next_pred.detach().numpy()
  #sim_length=1

RuntimeError: Error(s) in loading state_dict for E2C:
	Missing key(s) in state_dict: "trans.net.6.weight", "trans.net.6.bias", "trans.net.7.weight", "trans.net.7.bias", "trans.net.7.running_mean", "trans.net.7.running_var", "trans.net.9.weight", "trans.net.9.bias", "trans.net.10.weight", "trans.net.10.bias", "trans.net.10.running_mean", "trans.net.10.running_var", "trans.net.12.weight", "trans.net.12.bias", "trans.net.13.weight", "trans.net.13.bias", "trans.net.13.running_mean", "trans.net.13.running_var". 
	size mismatch for encoder.net.15.weight: copying a param with shape torch.Size([16, 150]) from checkpoint, the shape in current model is torch.Size([4, 150]).
	size mismatch for encoder.net.15.bias: copying a param with shape torch.Size([16]) from checkpoint, the shape in current model is torch.Size([4]).
	size mismatch for decoder.net.0.weight: copying a param with shape torch.Size([150, 8]) from checkpoint, the shape in current model is torch.Size([150, 2]).
	size mismatch for tr

In [31]:
t=springTransition(8,1)